In [2]:
import sys
!{sys.executable} -m pip install fredapi yfinance alpha_vantage

  Using cached fredapi-0.5.2-py3-none-any.whl.metadata (5.0 kB)
  Using cached yfinance-1.0-py2.py3-none-any.whl.metadata (6.0 kB)
  Using cached alpha_vantage-3.0.0-py3-none-any.whl.metadata (12 kB)
  Using cached multitasking-0.0.12.tar.gz (19 kB)
  Installing build dependencies ... one
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached peewee-3.19.0-py3-none-any.whl.metadata (7.0 kB)
  Using cached curl_cffi-0.13.0-cp39-abi3-macosx_11_0_arm64.whl.metadata (13 kB)
  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiosignal-1.4.0-py3-none-any.whl.metadata (3.7 kB)
Using cached fredapi-0.5.2-py3-none-any.whl (11 kB)
Using cached yfinance-1.0-py2.py3-none-any.whl (127 kB)
Using cached curl_cffi-0.13.0-cp39-abi3-macosx_11_0_arm64.whl (3.0 MB)
Using cached alpha_vantage-3.0.0-py3-none-any.whl (35 kB)
Using cached peewee-3.19.0-py3-none-any.whl (411 kB)
Using cached aiohappyeyeballs-2.6.1-py3-no

# MACRO DATA

In [11]:
from fredapi import Fred
import pandas as pd

fred = Fred(api_key='b29fa8db713c7d9498fc161a88fb77ac')

#UNRATE: unemployment rate
#CPIAUCSL: consumer price index (inflaton)
#FEDFUNDS: effective federal funds rate (interest rates)
#GDPC1: Real Gross Domestic Product
#T10Y2Y: Yield Curve (recession indicator)
#TEDRATE: TED Spread (interbank credit risk)
#VIXLS: CBOE (market fear)
#BAMLH0A0HYM2: Bank of America's US High Yield Index (Credit risk)

indexes = ['UNRATE', 'CPIAUCSL', 'FEDFUNDS', 'GDPC1', 'T10Y2Y', 'TEDRATE', 'VIXCLS', 'BAMLH0A0HYM2']

data = {}
for index in indexes:
    data[index] = fred.get_series(index)

df_macro = pd.DataFrame(data)
df_macro.index.name = 'Date'
df_macro = df_macro.loc['2000-01-01':]

In [12]:
df_macro

,UNRATE,CPIAUCSL,FEDFUNDS,GDPC1,T10Y2Y,TEDRATE,VIXCLS
Date,,,,,,,
2000-01-01,4.0,169.3,5.45,13878.147,NaN,NaN,NaN
2000-01-03,NaN,NaN,NaN,NaN,0.20,NaN,24.21
2000-01-04,NaN,NaN,NaN,NaN,0.19,0.77,27.01
2000-01-05,NaN,NaN,NaN,NaN,0.24,0.75,26.41
2000-01-06,NaN,NaN,NaN,NaN,0.22,0.78,25.73
...,...,...,...,...,...,...,...
2026-01-15,NaN,NaN,NaN,NaN,0.61,NaN,15.84
2026-01-16,NaN,NaN,NaN,NaN,0.65,NaN,15.86
2026-01-19,NaN,NaN,NaN,NaN,NaN,NaN,18.84


# Sector Specific

In [13]:
import yfinance as yf

#ticker symbols
# CL=F: crude oil futures
# GC=F: gold futures 
# XLE: energy select sector (correlation with oil)
# XLK: technology select sector
# XLF: financial sector (rate sensistive)
# GSPC: S and P 500 Index
# TNX: 10 year treasuring yield

tickers = ['CL=F', 'GC=F', 'XLE', 'XLK', 'XLF', '^GSPC', '^TNX']
df_sector = yf.download(tickers, start="2000-01-01", interval="1d", auto_adjust=True)['Close']
df_sector

[*********************100%***********************]  7 of 7 completed


Ticker,CL=F,GC=F,XLE,XLF,XLK,^GSPC,^TNX
Date,,,,,,,
2000-01-03,NaN,NaN,6.699405,11.147916,20.652033,1455.219971,6.548
2000-01-04,NaN,NaN,6.573298,10.660573,19.604296,1399.420044,6.485
2000-01-05,NaN,NaN,6.746695,10.576816,19.313257,1402.109985,6.599
2000-01-06,NaN,NaN,7.006791,11.041307,18.672983,1403.449951,6.549
2000-01-07,NaN,NaN,7.081666,11.224066,18.998940,1441.469971,6.504
...,...,...,...,...,...,...,...
2026-01-14,62.020000,4626.299805,48.060001,54.150002,144.699997,6926.600098,4.140
2026-01-15,59.189999,4616.299805,47.610001,54.369999,145.460007,6944.470215,4.160
2026-01-16,59.439999,4588.399902,47.689999,54.439999,145.619995,6940.009766,4.231


# global interdependencies

In [15]:
from alpha_vantage.foreignexchange import ForeignExchange 
import time

fe = ForeignExchange(key='DCCXHV23S5TTXJT1', output_format='pandas')

#exchange rates for US dollar against other
currencies = ['EUR', 'KRW', 'CAD']
fx_data = []

for currency in currencies:
    try:
        data, _ = fe.get_currency_exchange_daily(from_symbol='USD', to_symbol=currency)
        
        series = data['4. close'].rename(f'USD_{currency}')
        fx_data.append(series)
        
        # Pause to respect free tier API limits (5 calls per minute)
        time.sleep(3) 
        
    except Exception as e:
        print(f"Error fetching {currency}: {e}")

df_fx = pd.concat(fx_data, axis=1)

In [16]:
df_fx

,USD_EUR,USD_KRW,USD_CAD
date,,,
2025-09-03,NaN,1390.13,1.3793
2025-09-04,0.8584,1393.40,1.3818
2025-09-05,0.8532,1386.18,1.3829
2025-09-08,0.8499,1385.08,1.3800
2025-09-09,0.8540,1389.03,1.3842
...,...,...,...
2026-01-15,0.8613,1469.97,1.3891
2026-01-16,0.8620,1473.29,1.3915
2026-01-19,0.8586,1473.70,1.3865


In [9]:
from alpha_vantage.foreignexchange import ForeignExchange 

fe = ForeignExchange(key='DCCXHV23S5TTXJT1', output_format='pandas')

df = fe.get_currency_exchange_daily(from_symbol='USD', to_symbol='EUR')
df

(            1. open  2. high  3. low  4. close
 date                                          
 2026-01-14   0.8586   0.8594  0.8575    0.8587
 2026-01-13   0.8569   0.8595  0.8562    0.8588
 2026-01-12   0.8590   0.8604  0.8548    0.8570
 2026-01-09   0.8575   0.8607  0.8573    0.8592
 2026-01-08   0.8565   0.8589  0.8558    0.8576
 ...             ...      ...     ...       ...
 2025-09-03   0.8590   0.8614  0.8558    0.8575
 2025-09-02   0.8538   0.8610  0.8532    0.8589
 2025-09-01   0.8556   0.8557  0.8519    0.8538
 2025-08-29   0.8559   0.8582  0.8539    0.8556
 2025-08-28   0.8591   0.8598  0.8547    0.8559
 
 [100 rows x 4 columns],
 {'1. Information': 'Forex Daily Prices (open, high, low, close)',
  '2. From Symbol': 'USD',
  '3. To Symbol': 'EUR',
  '4. Output Size': 'Compact',
  '5. Last Refreshed': '2026-01-14',
  '6. Time Zone': 'UTC'})

# EDA

In [17]:
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller

In [18]:
df_main = df_sector.join(df_macro, how='outer').join(df_fx, how='outer')
df_main.sort_index(inplace=True)

# forward fill bc market uses that monthly or weekly number
df_main.fillna(method='ffill', inplace=True)
df_main.dropna(inplace=True)
df_main

/var/folders/n9/hrwgbvx936vf2n8y560vb_lc0000gn/T/ipykernel_52928/2310242047.py:5: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_main.fillna(method='ffill', inplace=True)


,CL=F,GC=F,XLE,XLF,XLK,^GSPC,^TNX,UNRATE,CPIAUCSL,FEDFUNDS,GDPC1,T10Y2Y,TEDRATE,VIXCLS,USD_EUR,USD_KRW,USD_CAD
2025-09-04,63.480000,3565.800049,43.789852,53.694958,130.952820,6502.080078,4.176,4.4,324.368,4.22,24024.957,0.58,0.09,15.30,0.8584,1393.40,1.3818
2025-09-05,61.869999,3613.199951,42.944336,52.711639,131.062500,6481.500000,4.086,4.4,324.368,4.22,24024.957,0.59,0.09,15.18,0.8532,1386.18,1.3829
2025-09-08,62.259998,3638.100098,42.846020,52.751373,132.044739,6495.149902,4.046,4.4,324.368,4.22,24024.957,0.56,0.09,15.11,0.8499,1385.08,1.3800
2025-09-09,62.630001,3643.300049,43.052486,52.979820,132.299042,6512.609863,4.074,4.4,324.368,4.22,24024.957,0.54,0.09,15.04,0.8540,1389.03,1.3842
2025-09-10,63.669998,3643.600098,43.804600,52.801033,134.697311,6532.040039,4.032,4.4,324.368,4.22,24024.957,0.50,0.09,15.35,0.8549,1389.07,1.3862
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2026-01-15,59.189999,4616.299805,47.610001,54.369999,145.460007,6944.470215,4.160,4.4,326.030,3.72,24024.957,0.61,0.09,15.84,0.8613,1469.97,1.3891
2026-01-16,59.439999,4588.399902,47.689999,54.439999,145.619995,6940.009766,4.231,4.4,326.030,3.72,24024.957,0.65,0.09,15.86,0.8620,1473.29,1.3915
2026-01-19,59.439999,4588.399902,47.689999,54.439999,145.619995,6940.009766,4.231,4.4,326.030,3.72,24024.957,0.65,0.09,18.84,0.8586,1473.70,1.3865
2026-01-20,60.340000,4759.600098,47.599998,53.200001,141.839996,6796.859863,4.295,4.4,326.030,3.72,24024.957,0.70,0.09,20.09,0.8529,1480.82,1.3835


In [ ]:
# need to make sure PCMCI picks up on actual dependencies, and not just a general growth

df_transform = pd.DataFrame(index=master_df.index)

#1. rates and spreads: these variables are percentages or indices so we care more about the change 

#2. asset prices: grows exponentially and would cause heteroscedasticity so use log 



#EDA
# assumption: PCMCI assumption that causal rules do not change over time

# CREDIT RISK

In [ ]:
FRED